In [1]:
import json
from datetime import datetime
import PyPDF2
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

import os
import re 
import math
import operator
import requests

from bs4 import BeautifulSoup as soup
from bs4.element import Comment

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import Counter
from pprint import pprint

In [2]:
pdf_file = open('FAQs.pdf', 'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)
number_of_pages = read_pdf.getNumPages()

In [3]:
questions = []
answers = []
for i in range(3, number_of_pages-1):
    page = read_pdf.getPage(i)
    page_content = page.extractText()
    
    while page_content.find('Q.')!=-1:
        question = page_content[page_content.find(' Q.')+4:page_content.find('A.')]
        questions.append(question.replace('\n',''))
        page_content = page_content[page_content.find('A.'):]
        answer = page_content[page_content.find('A.')+4:page_content.find('Q.')]
        answers.append(answer.replace('\n',''))
        page_content = page_content[page_content.find('A.')+4:]

In [4]:
print(questions)
print(answers)

['What are the advantages of HPE OneView? ', 'What is a software-defined approach to lifecycle management? ', 'Can I customize functionality and integrate HPE OneView with my existing tools and environment? ', 'Does HPE provide integration kits for customers who do not wish to develop their own? ', 'Is HPE OneView a refresh to existing HPE infrastructure management tools or a brand-new design? ', 'What capabilities does HPE OneView offer? ', 'How is HPE OneView licensed? ', 'What server platforms are supported by HPE OneView 5.0? ', 'Can I create a template from an existing profile? ', 'Does HPE OneView support granular access control? ', 'What is two-factor authentication and is it supported by HPE OneView? ', 'What is an SPP? ', 'Does HPE OneView use SPPs to update the firmware and driver of servers? ', 'Does HPE OneView support external repository to store SPPs? ', 'Can you schedule the activation of firmware and OS driver updates? ', 'Does HPE OneView allow online profile configura

In [46]:
# Read text from Documents
# Tokenize data

words = []
doc_text = []
stop_words = set(stopwords.words('english'))


for q in questions:
    
    
    # Tokenize
    
    tokens = word_tokenize(q)
    
    # Stop word removal
    
    filtered_words = [w for w in tokens if not w in stop_words]
    
    new_words = []
    ps = PorterStemmer() 
    for w in filtered_words:
        if len(w) != 1:
            new_words.append(ps.stem(w))
    doc_text.append(new_words)
    
    for w in new_words:
        words.append(w)
        
print('Word count: ', len(words))

Word count:  541


In [47]:
# Word frequency

frequency = Counter(words)
print('Word frequency: \n')
print(frequency)

Word frequency: 

Counter({'hpe': 63, 'oneview': 49, 'support': 29, 'doe': 24, 'what': 20, 'how': 12, 'manag': 11, 'can': 10, 'remot': 9, 'global': 8, 'dashboard': 8, 'use': 7, 'technician': 6, 'Is': 5, 'firmwar': 5, 'integr': 4, 'tool': 4, 'licens': 4, 'server': 4, 'access': 4, 'updat': 4, 'driver': 4, 'configur': 4, 'san': 4, 'custom': 3, 'exist': 3, 'provid': 3, '5.0': 3, 'profil': 3, 'spp': 3, 'extern': 3, 'storag': 3, 'brocad': 3, 'autom': 3, 'need': 3, 'switch': 3, 'advanc': 3, 'insight': 3, 'avail': 3, 'will': 3, 'environ': 2, 'offer': 2, 'control': 2, 'schedul': 2, 'inventori': 2, 'ipv6': 2, 'FC': 2, 'data': 2, 'volum': 2, 'provis': 2, 'storevirtu': 2, 'virtual': 2, 'connect': 2, 'mani': 2, 'interconnect': 2, 'modul': 2, 'top-of-rack': 2, 'tor': 2, 'microsoft': 2, 'plug-in': 2, 'releas': 2, 'vmware': 2, 'deliv': 2, 'system': 2, 'enterpris': 2, 'statu': 2, 'track': 2, 'requir': 2, 'time': 2, 'advantag': 1, 'software-defin': 1, 'approach': 1, 'lifecycl': 1, 'function': 1, 'kit': 

In [21]:
# Cosine similarity

WORD = re.compile(r'\w+')

# Calculate the cosine similarity between the two vectors
# Parameters:
#     vec1: Vector 1
#     vec2: Vector 2
# Return: 
#     Cosine similarity value
def get_cosine(vec1, vec2):
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

# Vectorize words
# Parameters:
#     text: Text data
# Return:
#     Text vector
def text_to_vector(text):
    
    words = WORD.findall(text)
    return Counter(words)

In [84]:
userQUERY = "SPP stands for?"
QUERY = userQUERY.replace("?", "").split(" ")
filtered_query = [ps.stem(w) for w in QUERY if not w in stop_words]
print(filtered_query)

['spp', 'stand']


In [85]:
cosine_similarities = {}

for idx, data in enumerate(doc_text):
    
    data = ' '.join(data)
    
    vector1 = text_to_vector(" ".join(filtered_query))
    vector2 = text_to_vector(data)
    
    cosine_similarities[idx] = get_cosine(vector1, vector2)

    
key_max = max(cosine_similarities.keys(), key=(lambda k: cosine_similarities[k]))

print('Maximum Value: ', cosine_similarities[key_max], '\n')
print("Q. ", userQUERY)

if(cosine_similarities[key_max] < 0.4):
    print("A. ", "Kindly be more precise.")
else:
    print("A. ", answers[key_max])

Maximum Value:  0.4999999999999999 

Q.  SPP stands for?
A.  The Service Pack for ProLiant (SPP) is a comprehensive ISO image package that includes firmware, drivers, and tools for HPE ProLiant servers and infrastructure, including a broad range of multi-generational HPE ProLiant servers. 
